In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
%matplotlib inline
pylab.rcParams[ 'figure.figsize' ] = 10 , 8
from sklearn import preprocessing

In [3]:
Hotel_Reviews=pd.read_csv("Hotel_Reviews.csv",usecols=['Hotel_Name','Positive_Review','Negative_Review'])
TEMP=Hotel_Reviews.loc[Hotel_Reviews['Hotel_Name']=='Hotel Arena']
TEMP.head()

,Hotel_Name,Negative_Review,Positive_Review
0,Hotel Arena,I am so angry that i made this post available...,Only the park outside of the hotel was beauti...
1,Hotel Arena,No Negative,No real complaints the hotel was great great ...
2,Hotel Arena,Rooms are nice but for elderly a bit difficul...,Location was good and staff were ok It is cut...
3,Hotel Arena,My room was dirty and I was afraid to walk ba...,Great location in nice surroundings the bar a...
4,Hotel Arena,You When I booked with your company on line y...,Amazing location and building Romantic setting


In [4]:
import langid
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/LiverJoker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
corpusP= []
corpusN= []
uniqueUsefulTokens= set()

for i in TEMP.index:
       reviewP=""
       reviewN="" 
       if ("No Negative" in TEMP['Negative_Review'][i]) and "No Positive" in TEMP['Positive_Review'][i]:
            continue
       elif "No Negative" in TEMP['Negative_Review'][i]:
            reviewP = TEMP['Positive_Review'][i]
       else:
            reviewN = TEMP['Negative_Review'][i]
       
       if (langid.classify(reviewP)[0]!='en' and langid.classify(reviewN)[0]!='en'):
            continue
       reviewP= re.sub('[^a-zA-Z]', ' ', reviewP)
       reviewN= re.sub('[^a-zA-Z]', ' ', reviewN)

       reviewP=reviewP.lower().split()
       reviewN=reviewN.lower().split()

       ps=PorterStemmer()
       reviewP  = [ps.stem(word) for word in reviewP if not word in set(stopwords.words('english'))]
       reviewN  = [ps.stem(word) for word in reviewN if not word in set(stopwords.words('english'))]
       for word in reviewP:
            uniqueUsefulTokens.add(word)
       reviewP = ' '.join(reviewP)
    
       for word in reviewN:
            uniqueUsefulTokens.add(word)
       reviewN = ' '.join(reviewN)
    
       if reviewP!="":
            corpusP.append(reviewP)
        
    
       if reviewN!="":
            corpusN.append(reviewN)
            
    

In [18]:
len(corpusP)

91

In [19]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

def computeIDF(docList):
    import math
    ifdDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(),0)
    for doc in docList:
        for word, val in doc.items():
            if val>0:
                idfDict[word] +=1
            
    for word, val in idfDict.items():
        idfDict[word] = math.log(N/float(val))
        
    return idfDict

def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [20]:
from functools import reduce

In [21]:
bowA = []
for line in corpusN:
    line= re.sub('[^a-zA-Z]', ' ', line)
    bowA.append(line.split(" "))

bowAnew = reduce(lambda x,y: x+y,bowA)
len(bowA)

314

In [22]:
wordSet=set(bowAnew)
wordSet

{'board',
 'transpar',
 'scaffold',
 'hole',
 'walkabl',
 'suppos',
 'min',
 'computer',
 'heater',
 'reach',
 'creat',
 'aircondit',
 'aspect',
 'insid',
 'rodent',
 'british',
 'varieti',
 'black',
 'nearbi',
 'cold',
 'person',
 'ineffect',
 'air',
 'damag',
 'tax',
 'neighbor',
 'expans',
 'tortur',
 'watch',
 'fuss',
 'round',
 'stop',
 'plu',
 'lift',
 'drip',
 'arriv',
 'tasteless',
 'underway',
 'weapon',
 'stranger',
 'handl',
 'bother',
 'upper',
 'darken',
 'littl',
 'kingsiz',
 'market',
 'restor',
 'opportun',
 'hospit',
 'photo',
 'remedi',
 'need',
 'huge',
 'unsaf',
 'hoover',
 'par',
 'ordinari',
 'dutch',
 'attempt',
 'metro',
 'elderli',
 'compani',
 'town',
 'prepaid',
 'iron',
 'surround',
 'stori',
 'alreadi',
 'metal',
 'suitabl',
 'knock',
 'electr',
 'situat',
 'soak',
 'develop',
 'sofa',
 'stuffi',
 'love',
 'produc',
 'went',
 'femal',
 'function',
 'spent',
 'monday',
 'readi',
 'enough',
 'heineken',
 'half',
 'deliv',
 'minor',
 'flow',
 'robberi',
 'some

In [23]:
wordDictAarr = np.array([])
for listbowA in bowA:
    wordDictA = dict.fromkeys(wordSet,0)
    for word in listbowA:
        wordDictA[word]+=1
    wordDictAarr = np.append(wordDictAarr, wordDictA)


In [24]:
i = 0
tfBowAarr = np.array([])
for listbowA in bowA:
    tfBowA = computeTF(wordDictAarr[i], listbowA)
    i +=1
    tfBowAarr = np.append(tfBowAarr, tfBowA)
tfBowAarr[0]

{'board': 0.0,
 'transpar': 0.0,
 'scaffold': 0.0,
 'hole': 0.0,
 'walkabl': 0.0,
 'suppos': 0.0,
 'min': 0.0,
 'computer': 0.0,
 'heater': 0.0,
 'reach': 0.0,
 'creat': 0.0,
 'aircondit': 0.0,
 'aspect': 0.0,
 'insid': 0.0,
 'rodent': 0.0,
 'british': 0.0,
 'varieti': 0.0,
 'black': 0.0,
 'nearbi': 0.0,
 'cold': 0.0,
 'person': 0.0,
 'ineffect': 0.0,
 'air': 0.0,
 'damag': 0.0,
 'tax': 0.0,
 'neighbor': 0.0,
 'expans': 0.0,
 'tortur': 0.0,
 'watch': 0.0,
 'fuss': 0.0,
 'round': 0.0,
 'stop': 0.005847953216374269,
 'plu': 0.0,
 'lift': 0.0,
 'drip': 0.0,
 'arriv': 0.005847953216374269,
 'tasteless': 0.0,
 'underway': 0.0,
 'weapon': 0.005847953216374269,
 'stranger': 0.0,
 'handl': 0.0,
 'bother': 0.0,
 'upper': 0.0,
 'darken': 0.0,
 'littl': 0.0,
 'kingsiz': 0.0,
 'market': 0.0,
 'restor': 0.0,
 'opportun': 0.0,
 'hospit': 0.0,
 'photo': 0.0,
 'remedi': 0.0,
 'need': 0.0,
 'huge': 0.0,
 'unsaf': 0.0,
 'hoover': 0.0,
 'par': 0.0,
 'ordinari': 0.0,
 'dutch': 0.0,
 'attempt': 0.0,
 'metr

In [25]:
idfs = computeIDF(wordDictAarr)
idfsData = pd.DataFrame.from_dict(idfs, orient='index', dtype=float)
idfsData[0]=1/idfsData[0]

PosRev = pd.DataFrame(idfsData[0].sort_values(ascending=False))
PosRev.iloc[0:20, :]

,0
room,1.317592
hotel,0.793175
work,0.592084
bathroom,0.538337
construct,0.491202
bed,0.473528
go,0.455779
staff,0.449836
would,0.449836
breakfast,0.449836


In [14]:
tfidfBowA = np.array([])
for tfBowA in tfBowAarr:
    tfidf = computeTFIDF(tfBowA, idfs)
    tfidfBowA = np.append(tfidfBowA, tfidf)
tfidfBowA[7]

{'board': 0.0,
 'improved': 0.0,
 'twins': 0.0,
 'min': 0.0,
 'heater': 0.0,
 'reach': 0.0,
 'rodent': 0.0,
 'british': 0.0,
 'staircase': 0.0,
 'black': 0.0,
 'pointed': 0.0,
 'cold': 0.0,
 'table': 0.0,
 'attacked': 0.0,
 'air': 0.0,
 'person': 0.0,
 'resulted': 0.0,
 'flooded': 0.0,
 'tax': 0.0,
 'watch': 0.0,
 'fuss': 0.0,
 'round': 0.0,
 'stop': 0.0,
 'replacement': 0.0,
 'cleaning': 0.0,
 'advance': 0.0,
 'lift': 0.0,
 'bloking': 0.0,
 'guests': 0.0,
 'loading': 0.0,
 'information': 0.0,
 'details': 0.0,
 'pushing': 0.0,
 'tasteless': 0.0,
 'smelly': 0.0,
 'underway': 0.0,
 'hopefully': 0.0,
 'weapon': 0.0,
 'stranger': 0.0,
 'bother': 0.0,
 'upper': 0.0,
 'darken': 0.0,
 'covering': 0.0,
 'managent': 0.0,
 'market': 0.0,
 'rectified': 0.0,
 'lobby': 0.0,
 'potential': 0.0,
 'need': 0.0,
 'huge': 0.0,
 'changed': 0.0,
 'par': 0.0,
 'dutch': 0.0,
 'attempt': 0.0,
 'efficiently': 0.0,
 'metro': 0.0,
 'leaves': 0.0,
 'sticking': 0.0,
 'cleaniness': 0.0,
 'town': 0.0,
 'prepaid': 0.0

In [15]:
idfs = computeIDF([wordDictA])


ZeroDivisionError: float division by zero

In [ ]:
tfidfBowA = computeTFIDF(tfBowA, idfs)